In [9]:
pip install selenium

Note: you may need to restart the kernel to use updated packages.


# crawl vn express

In [10]:
import time

import pandas as pd

import os

import json

from selenium import webdriver

from selenium.webdriver.support.ui import WebDriverWait

from selenium.webdriver.common.by import By



def crawl_data_vnexpress_final(start_url, namefile, max_articles=100):

    """

    Hàm cào dữ liệu cốt lõi

    """

    # --- CẤU HÌNH ---

    options = webdriver.ChromeOptions()

    # options.add_argument("headless") # Bỏ comment dòng này nếu muốn chạy ẩn (không hiện trình duyệt)

    browser = webdriver.Chrome(options=options)

    

    # Tạo file nếu chưa có

    if not os.path.isfile(namefile + ".csv"):

        df = pd.DataFrame(columns=["title", "description", "body", "date", "url", "category", "tags", "author"])

        df.to_csv(namefile + ".csv", index=False, encoding="utf-8-sig")



    def is_ready(browser):

        return browser.execute_script("return document.readyState === 'complete'")



    # --- KHỞI TẠO BIẾN ĐẾM ---

    current_page_url = start_url

    total_crawled = 0 

    page_count = 0    



    print(f"🚀 [START] Bắt đầu cào chủ đề: {namefile}. Mục tiêu: {max_articles} bài.")



    try:

        while current_page_url:

            # Kiểm tra ngay đầu vòng lặp lớn

            if total_crawled >= max_articles:

                print(f"✅ Đã đạt đủ chỉ tiêu {max_articles} bài. Dừng chủ đề này.")

                break



            page_count += 1

            print(f"\n--- 📄 Đang vào trang danh mục thứ {page_count}: {current_page_url} ---")

            

            try:

                browser.get(current_page_url)

                WebDriverWait(browser, 30).until(is_ready)

            except Exception as e:

                print(f"Lỗi load trang danh mục: {e}")

                break

            

            # 1. LẤY LIST LINK

            article_links = []

            try:

                article_elements = browser.find_elements(By.CSS_SELECTOR, ".title-news a")

                for el in article_elements:

                    try:

                        href = el.get_attribute("href")

                        if href and "vnexpress.net" in href and "video" not in href and "podcast" not in href:

                             article_links.append(href)

                    except: continue

                # Lọc trùng trong trang

                article_links = list(dict.fromkeys(article_links))

            except:

                print("Không tìm thấy bài viết nào ở trang này.")

                

            print(f"--> Tìm thấy {len(article_links)} link trên trang này.")



            # 2. LẤY LINK TRANG KẾ TIẾP (Để dành)

            next_page_url = None

            try:

                next_btns = browser.find_elements(By.XPATH, "//a[contains(@class, 'next-page')]")

                if next_btns:

                    next_page_url = next_btns[0].get_attribute("href")

            except: pass



            # 3. VÀO TỪNG BÀI

            for link in article_links:

                # Kiểm tra điều kiện dừng

                if total_crawled >= max_articles:

                    break 



                # Check trùng file CSV

                is_exist = False

                try:

                    if os.path.exists(namefile + ".csv"):

                        # Đọc file CSV chỉ lấy cột url để check cho nhanh

                        existing_df = pd.read_csv(namefile + ".csv", usecols=['url'])

                        if link in existing_df['url'].values:

                            # print(f"⚠️ Bỏ qua (Đã tồn tại): {link}")

                            is_exist = True

                except: pass

                

                if is_exist: continue



                # --- BẮT ĐẦU CÀO ---

                try:

                    browser.get(link)

                    WebDriverWait(browser, 10).until(is_ready)

                    

                    # ... GET DATA ...

                    try: title = browser.find_element(By.XPATH, "//h1[contains(@class, 'title-detail')]").text

                    except: title = ""



                    try: description = browser.find_element(By.XPATH, "//p[contains(@class, 'description')]").text

                    except: 

                        try: description = browser.find_element(By.XPATH, "//div[@class='lead_detail']").text

                        except: description = ""



                    str_body = ""

                    try:

                        paras = browser.find_elements(By.XPATH, "//p[@class='Normal']")

                        for p in paras: str_body += p.text + "\n"

                    except: pass

                    if str_body == "": str_body = description



                    try: date = browser.find_element(By.XPATH, "//span[@class='date']").text

                    except: date = ""



                    try: category = browser.find_element(By.XPATH, "//ul[contains(@class, 'breadcrumb')]/li/a[contains(@data-medium, 'Menu-')][1]").text

                    except: category = "None"



                    try:

                        tags_elems = browser.find_elements(By.XPATH, "//div[contains(@class, 'tags')]//a")

                        list_tags = [t.text for t in tags_elems if t.text]

                        str_tags = json.dumps(list_tags, ensure_ascii=False)

                    except: str_tags = "[]"



                    try: author = browser.find_element(By.XPATH, "//p[@class='Normal' and @style='text-align:right;']").text

                    except: author = "None"

                    

                    # ... SAVE DATA ...

                    new_row = pd.DataFrame([[title, description, str_body, date, link, category, str_tags, author]], 

                                           columns=["title", "description", "body", "date", "url", "category", "tags", "author"])

                    new_row.to_csv(namefile + ".csv", mode='a', index=False, encoding="utf-8-sig", header=False)

                    

                    total_crawled += 1

                    print(f"💾 [{total_crawled}/{max_articles}] Saved: {title[:30]}...")



                except Exception as e:

                    print(f"❌ Lỗi link {link}: {e}")

            

            # 4. CHUYỂN TRANG

            if next_page_url and total_crawled < max_articles:

                current_page_url = next_page_url

            else:

                print("Hết trang để chuyển.")

                break 

                

    except Exception as e:

        print(f"Lỗi hệ thống: {e}")

    finally:

        browser.quit()

        print(f"🎉 HOÀN THÀNH CHỦ ĐỀ: {namefile}! Tổng số bài: {total_crawled}")





# ============================================================

# --- MASTER SCRIPT: CHẠY TỰ ĐỘNG 4 CHỦ ĐỀ LIÊN TIẾP ---

# ============================================================



def run_project_classification():

    # Cấu hình danh sách các topic cần cào

    # Format: (URL chuyên mục, Tên file CSV, Số lượng bài)

    tasks = [

        ("https://vnexpress.net/the-thao", "data_vnexpress_thethao", 1000),

        ("https://vnexpress.net/suc-khoe", "data_vnexpress_suckhoe", 1000),

        ("https://vnexpress.net/giai-tri", "data_vnexpress_giaitri", 1000),

        ("https://vnexpress.net/khoa-hoc", "data_vnexpress_khoahoc", 1000)

    ]



    print("\n" + "="*50)

    print("🤖 BẮT ĐẦU CHIẾN DỊCH CRAWL DATA CLASSIFICATION")

    print("="*50 + "\n")

    

    for url, filename, limit in tasks:

        print(f"\n👉 Đang xử lý: {filename.upper()}...")

        try:

            # Gọi hàm crawl cho từng topic

            crawl_data_vnexpress_final(url, filename, max_articles=limit)

            

            print(f"✅ Xong {filename}. Nghỉ 10 giây để bảo vệ IP...")

            time.sleep(10) # Nghỉ giữa các topic để tránh bị chặn

            

        except Exception as e:

            print(f"❌ Có lỗi nghiêm trọng khi chạy {filename}: {e}")

            print("Chuyển sang topic tiếp theo...")

            continue



    print("\n" + "="*50)

    print("🎉🎉🎉 ĐÃ HOÀN TẤT TOÀN BỘ PROJECT! KIỂM TRA 4 FILE CSV.")

    print("="*50)



# Chạy chương trình

run_project_classification()


🤖 BẮT ĐẦU CHIẾN DỊCH CRAWL DATA CLASSIFICATION


👉 Đang xử lý: DATA_VNEXPRESS_THETHAO...
🚀 [START] Bắt đầu cào chủ đề: data_vnexpress_thethao. Mục tiêu: 1000 bài.

--- 📄 Đang vào trang danh mục thứ 1: https://vnexpress.net/the-thao ---
--> Tìm thấy 50 link trên trang này.
💾 [1/1000] Saved: Vì sao Chelsea nhanh chóng sa ...
💾 [2/1000] Saved: Guardiola: 'Man City không ghi...
💾 [3/1000] Saved: 'Kỷ nguyên bong bóng' của bóng...
💾 [4/1000] Saved: Cầu thủ hay nhất giải U11, U13...
💾 [5/1000] Saved: Cầu thủ hay nhất giải U11, U13...
💾 [6/1000] Saved: 8 điểm nhấn ở vòng 19 Ngoại hạ...
💾 [7/1000] Saved: 8 điểm nhấn ở vòng 19 Ngoại hạ...
💾 [8/1000] Saved: Ngoại hạng Anh nhận sai khi từ...
💾 [9/1000] Saved: Ngoại hạng Anh nhận sai khi từ...
💾 [10/1000] Saved: 11 tài năng trẻ hứa hẹn bùng n...
💾 [11/1000] Saved: 11 tài năng trẻ hứa hẹn bùng n...
💾 [12/1000] Saved: Cựu danh thủ Italy dẫn Arab Sa...
💾 [13/1000] Saved: Gabriel Jesus: 'Gã khổng lồ sa...
💾 [14/1000] Saved: Venus Williams nhận vé đặc 

# crawl dan tri

In [12]:
import time

import pandas as pd

import os

from selenium import webdriver

from selenium.webdriver.support.ui import WebDriverWait

from selenium.webdriver.common.by import By

from selenium.webdriver.support import expected_conditions as EC



# ==============================================================================

# HÀM CRAWL CỐT LÕI (CORE FUNCTION)

# ==============================================================================

def crawl_dantri_topic_final(base_url, namefile, target_count=1000):



    # --- 1. TỰ ĐỘNG LẤY SLUG ĐỂ LỌC LINK RÁC ---

    # Ví dụ: link '.../suc-khoe.htm' -> slug là '/suc-khoe/'

    # Chỉ bấm vào các link chứa slug này để tránh click nhầm sang chuyên mục khác

    clean_slug = base_url.split("/")[-1].replace(".htm", "")

    topic_slug = f"/{clean_slug}/"



    print("="*60)

    print(f"🚀 [START] BẮT ĐẦU CÀO CHỦ ĐỀ: {clean_slug.upper()}")

    print(f"🎯 Mục tiêu: {target_count} bài viết")

    print(f"📂 File lưu: {namefile}.csv")

    print("="*60)



    # --- CẤU HÌNH BROWSER ---

    options = webdriver.ChromeOptions()

    # options.add_argument("headless") # Bỏ comment dòng này nếu muốn chạy ẩn (nhanh hơn)

    options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36")

    options.add_argument("--disable-blink-features=AutomationControlled")



    # Tắt tải ảnh để tăng tốc độ cào (rất quan trọng khi cào 1000 bài)

    prefs = {"profile.managed_default_content_settings.images": 2}

    options.add_experimental_option("prefs", prefs)



    browser = webdriver.Chrome(options=options)



    # --- KHỞI TẠO FILE CSV ---

    if not os.path.isfile(namefile + ".csv"):

        df = pd.DataFrame(columns=["title", "description", "body", "date", "url", "category", "tags", "source"])

        df.to_csv(namefile + ".csv", index=False, encoding="utf-8-sig")



    total_crawled = 0

    page_num = 1



    try:

        # VÒNG LẶP VÔ TẬN (Sẽ break khi đủ số lượng)

        while total_crawled < target_count:



            # --- XỬ LÝ PHÂN TRANG (PAGINATION) ---

            # Trang 1: url gốc

            # Trang 2 trở đi: thêm /trang-x.htm

            if page_num == 1:

                current_url = base_url

            else:

                base_clean = base_url.replace(".htm", "")

                current_url = f"{base_clean}/trang-{page_num}.htm"



            print(f"\n📄 ĐANG QUÉT TRANG {page_num}: {current_url}")

            print(f"📊 Tiến độ: {total_crawled}/{target_count} ({round(total_crawled/target_count*100, 1)}%)")



            try:

                browser.get(current_url)

                # Chờ danh sách bài viết hiện ra

                WebDriverWait(browser, 10).until(EC.presence_of_element_located((By.TAG_NAME, "h3")))

            except:

                print(f"❌ Lỗi load trang {page_num}. Bỏ qua sang trang kế.")

                page_num += 1

                continue



            # -----------------------------------------------------

            # GIAI ĐOẠN 1: LẤY LINK & LỌC NHIỄU

            # -----------------------------------------------------

            article_links = []

            try:

                # Lấy tất cả thẻ A có đuôi .htm

                elements = browser.find_elements(By.XPATH, "//a[contains(@href, '.htm')]")



                for el in elements:

                    href = el.get_attribute("href")

                    if href and "dantri.com.vn" in href:

                        # --- BỘ LỌC THÔNG MINH ---

                        # 1. Link phải chứa slug của chủ đề (VD: /the-thao/)

                        if topic_slug not in href: continue

                        # 2. Không lấy link phân trang (trang-2, trang-3...)

                        if "trang-" in href: continue

                        # 3. Độ dài link phải đủ lớn (tránh link rác/menu)

                        if len(href) > 45:

                            article_links.append(href)



                # Khử trùng lặp trên trang hiện tại

                article_links = list(dict.fromkeys(article_links))

                print(f"   -> Tìm thấy {len(article_links)} link tiềm năng.")



            except Exception as e:

                print(f"Lỗi lấy link: {e}")



            # -----------------------------------------------------

            # GIAI ĐOẠN 2: CÀO CHI TIẾT TỪNG BÀI

            # -----------------------------------------------------

            for link in article_links:

                # Kiểm tra đủ chỉ tiêu chưa

                if total_crawled >= target_count:

                    break



                # Kiểm tra link đã có trong CSV chưa (Tránh cào lại tốn thời gian)

                is_exist = False

                if os.path.exists(namefile + ".csv"):

                    try:

                        # Đọc file CSV, chỉ check cột url cho nhanh

                        existing = pd.read_csv(namefile + ".csv", usecols=['url'])

                        if link in existing['url'].values: is_exist = True

                    except: pass



                if is_exist: continue



                # --- BẮT ĐẦU VÀO BÀI ---

                try:

                    browser.get(link)

                    WebDriverWait(browser, 8).until(EC.presence_of_element_located((By.TAG_NAME, "h1")))



                    # --- TRÍCH XUẤT DỮ LIỆU ---



                    # 1. Title

                    try: title = browser.find_element(By.TAG_NAME, "h1").text.strip()

                    except: title = ""



                    # 2. Description (Sapo)

                    description = ""

                    try: description = browser.find_element(By.CLASS_NAME, "singular-sapo").text.strip()

                    except: pass



                    # 3. Body (Nội dung chính)

                    str_body = ""

                    try:

                        # Class chuẩn: singular-content

                        content_div = browser.find_element(By.CLASS_NAME, "singular-content")

                        paras = content_div.find_elements(By.TAG_NAME, "p")

                        for p in paras:

                            # Lấy text, bỏ qua các dòng trống

                            if len(p.text.strip()) > 0:

                                str_body += p.text.strip() + "\n"

                    except: pass



                    # Fallback cho bài E-magazine (Giao diện tạp chí)

                    if str_body == "":

                        try:

                            paras = browser.find_elements(By.XPATH, "//div[contains(@class, 'e-magazine__body')]//p")

                            for p in paras: str_body += p.text + "\n"

                        except: pass



                    # 4. Date & Category

                    date = ""

                    try: date = browser.find_element(By.CLASS_NAME, "author-time").text

                    except: pass



                    category = clean_slug # Lấy luôn tên slug làm category



                    # --- LƯU DỮ LIỆU ---

                    if len(str_body) > 50: # Chỉ lưu bài có nội dung

                        new_row = pd.DataFrame([[title, description, str_body, date, link, category, "[]", "Dantri"]],

                                               columns=["title", "description", "body", "date", "url", "category", "tags", "source"])

                        new_row.to_csv(namefile + ".csv", mode='a', index=False, encoding="utf-8-sig", header=False)



                        total_crawled += 1

                        print(f"💾 [{total_crawled}/{target_count}] OK: {title[:40]}...")



                except Exception:

                    continue # Nếu lỗi bài này thì bỏ qua, đi bài tiếp



            # --- CHUYỂN TRANG SAU KHI QUÉT HẾT LINK ---

            if total_crawled >= target_count:

                print(f"✅ ĐÃ HOÀN THÀNH CHỦ ĐỀ {clean_slug.upper()}!")

                break

            else:

                page_num += 1

                # Cơ chế an toàn: Nếu quét quá 100 trang mà vẫn không đủ bài thì dừng (tránh treo)

                if page_num > 100:

                    print("⚠️ Đã quét quá 100 trang. Dừng bắt buộc.")

                    break



    except Exception as e:

        print(f"Lỗi hệ thống nghiêm trọng: {e}")

    finally:

        browser.quit()

        print(f"🎉 Kết thúc phiên làm việc cho chủ đề: {clean_slug}")





# ==============================================================================

# MASTER SCRIPT: CHẠY TỰ ĐỘNG TẤT CẢ TOPIC

# ==============================================================================

def run_full_dantri_project():



    # Danh sách nhiệm vụ (URL, Tên file muốn lưu)

    # Lưu ý: Topic Khoa học của Dân Trí tên chuẩn là "khoa-hoc-cong-nghe"

    tasks = [

        ("https://dantri.com.vn/suc-khoe.htm", "data_dantri_suckhoe"),

        ("https://dantri.com.vn/the-thao.htm", "data_dantri_thethao"),

        ("https://dantri.com.vn/giai-tri.htm", "data_dantri_giaitri"),

        ("https://dantri.com.vn/khoa-hoc.htm", "data_dantri_khoahoc")

    ]



    # Số lượng bài muốn lấy cho MỖI chủ đề

    TARGET_PER_TOPIC = 1000



    print("\n" + "#"*50)

    print("🤖 BẮT ĐẦU CHIẾN DỊCH CRAWL DÂN TRÍ - BIG DATA")

    print("#"*50 + "\n")



    for url, filename in tasks:

        try:

            # Gọi hàm crawl cho từng topic

            crawl_dantri_topic_final(url, filename, target_count=TARGET_PER_TOPIC)



            # Nghỉ ngơi giữa các topic để máy tính "thở" và tránh bị server chặn

            print("\n💤 Nghỉ 10 giây trước khi sang chủ đề tiếp theo...")

            time.sleep(10)



        except Exception as e:

            print(f"❌ Có lỗi khi chạy task {filename}: {e}")

            continue



    print("\n" + "#"*50)

    print("🎉🎉🎉 ĐÃ HOÀN TẤT TOÀN BỘ PROJECT! KIỂM TRA 4 FILE CSV.")

    print("#"*50)



# --- CHẠY CHƯƠNG TRÌNH ---

if __name__ == "__main__":

    run_full_dantri_project()


##################################################
🤖 BẮT ĐẦU CHIẾN DỊCH CRAWL DÂN TRÍ - BIG DATA
##################################################

🚀 [START] BẮT ĐẦU CÀO CHỦ ĐỀ: KHOA-HOC
🎯 Mục tiêu: 1000 bài viết
📂 File lưu: data_dantri_khoahoc.csv

📄 ĐANG QUÉT TRANG 1: https://dantri.com.vn/khoa-hoc.htm
📊 Tiến độ: 0/1000 (0.0%)
   -> Tìm thấy 26 link tiềm năng.
💾 [1/1000] OK: Trữ lượng dầu mỏ Venezuela chủ yếu là dầ...
💾 [2/1000] OK: Những lời tiên đoán của bà Vanga về năm ...
💾 [3/1000] OK: Thế giới động vật: Sư tử lao đến tấn côn...
💾 [4/1000] OK: Loạt khí tài "made in Việt Nam" nổi bật ...
💾 [5/1000] OK: Hiện tượng thiên văn hiếm gặp diễn ra đê...
💾 [6/1000] OK: Những hình ảnh khoa học nổi bật nhất tro...
💾 [7/1000] OK: Va chạm dữ dội trong hệ sao trẻ hé lộ gi...
💾 [8/1000] OK: Thư giãn trong ngày đầu năm với chùm ảnh...
💾 [9/1000] OK: UAV đa năng tầm xa và AI laptop thương h...
💾 [10/1000] OK: 100% dịch vụ công thiết yếu sẽ ứng dụng ...
💾 [11/1000] OK: Pháo hoa trong đêm giao th

In [17]:
import pandas as pd
df = pd.read_csv("/Users/tranquangtrong/Desktop/bds_DS/data_vnexpress_khoahoc.csv")
df.count()

title          625
description    625
body           625
date           615
url            630
category       615
tags           630
author         567
dtype: int64